In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import yfinance as yf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wd/'

Mounted at /content/drive


In [ ]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [ ]:
filename = 'PV_data.csv'
HV_data = pd.read_csv(path + filename)

In [ ]:
set_index(HV_data)

In [ ]:
tm_data = HV_data

In [ ]:
HV_data = HV_data.drop(columns=['PV'])

In [ ]:
HV_data['PV'] = tm_data['PV']

In [ ]:
HV_data

,"('GARCH', 'normal', 0)","('GARCH', 'gaussian', 0)","('GARCH', 'ged', 0)","('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)","('FIGARCH', 'gaussian', 0)","('FIGARCH', 'gaussian', 1)","('FIGARCH', 'ged', 0)","('FIGARCH', 'ged', 1)",gdp_growth,...,usd_eur,usd_jpy,usd_gbp,usd_cny,usd_cad,usd_mxn,gt_data,log_returns,"('APARCH', 'studentst', 1)",PV
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-02,0.039601,0.039601,0.040044,0.038388,0.038388,0.038388,0.038388,0.038865,0.038865,28984.929215,...,2.781970,307.252039,4.529620,23.415205,4.090860,26.750733,209.303607,-0.005604,0.037128,0.035334
2000-02-03,0.037979,0.037979,0.041000,0.038633,0.038633,0.038633,0.038633,0.040342,0.040342,28984.929215,...,2.799436,311.491144,4.529266,23.412659,4.119569,26.561052,209.303607,-0.008693,0.038336,0.036011
2000-02-04,0.036791,0.036791,0.038214,0.036990,0.036990,0.036990,0.036990,0.039265,0.039265,28984.929215,...,2.728548,306.262089,4.474254,23.415452,4.106629,26.433950,209.303607,0.000572,0.049778,0.034207
2000-02-07,0.046593,0.046593,0.047968,0.046474,0.046474,0.046474,0.046474,0.048168,0.048168,28984.929215,...,2.733357,299.491542,4.460147,23.414391,4.140641,26.339020,209.303607,0.027307,0.065732,0.038502
2000-02-08,0.049131,0.049131,0.049560,0.046711,0.046711,0.046711,0.046711,0.045562,0.045562,28984.929215,...,2.730104,301.761354,4.470789,23.415982,4.142338,26.135551,209.303607,0.010478,0.043373,0.041434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,0.000120,0.000120,0.000456,0.000121,0.000121,0.000121,0.000121,0.000423,0.000423,0.000000,...,-0.002546,0.509117,-0.003041,0.005162,0.001980,-0.105925,0.404061,0.008246,-0.004076,0.000011
2024-08-27,-0.003889,-0.003889,-0.003539,-0.004368,-0.004368,-0.004368,-0.004368,-0.001972,-0.001972,0.000000,...,0.002051,-0.551543,-0.001202,-0.006788,-0.001556,-0.173948,0.404061,0.009336,-0.005604,0.000029
2024-08-28,0.000280,0.000280,0.000449,0.000483,0.000483,0.000483,0.000483,0.000381,0.000381,0.000000,...,0.000849,0.247487,0.000849,0.002263,0.002051,-0.219910,0.707107,0.010307,-0.001642,0.000023


# Split train, validation and test data

In [ ]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

In [ ]:
def sliding_window_transform(X, y, timesteps):
  X_windows = []
  y_windows = []
  for i in range(len(X) - timesteps + 1):
    X_window = X[i:i + timesteps]
    y_window = y[i + timesteps - 1]
    X_windows.append(X_window)
    y_windows.append(y_window)

  return np.array(X_windows), np.array(y_windows)

In [ ]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

In [ ]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## PV

In [ ]:
daily_train, daily_val, daily_test = split_data(HV_data, 'PV')

In [ ]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [ ]:
HV_X_train, HV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
HV_X_val, HV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
HV_X_test, HV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [ ]:
# HV_X_train = HV_X_train.reshape(HV_X_train.shape[0], win_sz, -1)
# HV_X_val = HV_X_val.reshape(HV_X_val.shape[0], win_sz, -1)
# HV_X_test = HV_X_test.reshape(HV_X_test.shape[0], win_sz, -1)

### LSTM

In [ ]:
lstm_garch_model = Sequential()
lstm_garch_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(LSTM(16))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(Dense(5))

lstm_garch_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_garch_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 25, 48)              │          15,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,605 (76.58 KB)

 Trainable params: 19,605 (76.58 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lstm_history = lstm_garch_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - loss: 0.0073 - val_loss: 1.0608e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - loss: 1.0533e-04 - val_loss: 3.9712e-06
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 6.3497e-05 - val_loss: 4.1045e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - loss: 6.5936e-05 - val_loss: 2.7905e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 5.0899e-05 - val_loss: 1.8241e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 4.2152e-05 - val_loss: 7.1404e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 4.6088e-05 - val_loss: 6.5788e-07
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 4.3177e-05 - val_loss: 9.5981e-07
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 3.9228e-05 - val_loss: 1.5769e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 4.2683e-05 - val_loss: 2.6218e-04
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/ste

In [ ]:
# Predictions
HV_lstm_predictions = lstm_garch_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_lstm_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_lstm_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_lstm_predictions)
print(f'MAPE: {mape}%')

MSE: 2.611688609984893e-06
RMSE: 0.0016160719693085741
MAE: 0.0014694676902743827
MAPE: 6344.17074594108%


### GRU

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(GRU(16))
gru_model.add(Dropout(0.1))
gru_model.add(Dense(5))

gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
gru_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 25, 48)              │          11,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,917 (58.27 KB)

 Trainable params: 14,917 (58.27 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
gru_history = gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - loss: 0.0670 - val_loss: 6.2660e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 5.8746e-04 - val_loss: 4.2085e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 2.1113e-04 - val_loss: 1.8485e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - loss: 1.4633e-04 - val_loss: 1.0115e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - loss: 9.7844e-05 - val_loss: 6.6330e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - loss: 9.6379e-05 - val_loss: 8.1706e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - loss: 8.1637e-05 - val_loss: 4.4493e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 6.9791e-05 - val_loss: 3.6407e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 6.1209e-05 - val_loss: 3.9006e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 5.4266e-05 - val_loss: 3.9509e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/ste

In [ ]:
# Predictions
HV_gru_predictions = gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 2.8057435073858354e-06
RMSE: 0.0016750353749655066
MAE: 0.00133325639364019
MAPE: 5662.25831667539%


### LSTM_GRU

In [ ]:
lstm_gru_model = Sequential()
lstm_gru_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(GRU(16))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(Dense(5))

lstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

In [ ]:
lstm_gru_model.summary()

In [ ]:
lstm_gru_history = lstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - loss: 0.0109 - val_loss: 2.4173e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 2.6380e-04 - val_loss: 8.1992e-06
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 1.2662e-04 - val_loss: 3.1900e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - loss: 8.9400e-05 - val_loss: 5.8371e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 6.7880e-05 - val_loss: 3.4081e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - loss: 5.5710e-05 - val_loss: 3.2129e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - loss: 4.8164e-05 - val_loss: 1.4671e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - loss: 4.1400e-05 - val_loss: 4.8812e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - loss: 4.4342e-05 - val_loss: 2.2063e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 4.1704e-05 - val_loss: 5.9347e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step

In [ ]:
# Predictions
HV_lstm_gru_predictions = lstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_lstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 5.578489274671656e-07
RMSE: 0.0007468928487187205
MAE: 0.0005493840658028756
MAPE: 2212.6836840501746%


In [ ]:
# path_1 = '/content/drive/MyDrive/Volatility/Predictions/PV/'

# df_pred = pd.DataFrame(HV_lstm_gru_predictions[:,0], columns=['lstm_gru_pred'])
# filename = 'LSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)

### BiLSTM

In [ ]:
bilstm_model = Sequential()
bilstm_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Bidirectional(LSTM(16)))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Dense(5))

bilstm_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
bilstm_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ (None, 25, 96)              │          30,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 32)                  │          14,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 45,349 (177.14 KB)

 Trainable params: 45,349 (177.14 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
bilstm_history = bilstm_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - loss: 3.4415e-05 - val_loss: 4.2823e-06
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 17s 109ms/step - loss: 4.4700e-05 - val_loss: 1.4009e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - loss: 4.0948e-05 - val_loss: 8.2534e-07
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - loss: 3.4303e-05 - val_loss: 1.6793e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - loss: 3.0404e-05 - val_loss: 3.2688e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - loss: 3.9508e-05 - val_loss: 6.1384e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 13s 54ms/step - loss: 3.7375e-05 - val_loss: 1.9906e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 46ms/step - loss: 4.0867e-05 - val_loss: 5.9492e-08
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - loss: 3.9123e-05 - val_loss: 1.0652e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - loss: 2.9171e-05 - val_loss: 2.3213e-05
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s

In [24]:
# Predictions
HV_bilstm_predictions = bilstm_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [25]:
mse = mean_squared_error(HV_y_test, HV_bilstm_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_predictions)
print(f'MAPE: {mape}%')

MSE: 1.0247554957240659e-05
RMSE: 0.003201180244416215
MAE: 0.0031130404465647635
MAPE: 13608.89820555021%


### BiLSTM_GRU

In [ ]:
bilstm_gru_model = Sequential()
bilstm_gru_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(GRU(16))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(Dense(5))

bilstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
bilstm_gru_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional)      │ (None, 25, 96)              │          30,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 25, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 16)                  │           5,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 36,277 (141.71 KB)

 Trainable params: 36,277 (141.71 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
bilstm_gru_history = bilstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 13s 52ms/step - loss: 0.0323 - val_loss: 6.5646e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - loss: 4.2718e-04 - val_loss: 4.3528e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 1.5845e-04 - val_loss: 8.4139e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 45ms/step - loss: 1.0029e-04 - val_loss: 9.5934e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 46ms/step - loss: 6.8120e-05 - val_loss: 9.0690e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 5.3443e-05 - val_loss: 5.7811e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 51ms/step - loss: 4.5732e-05 - val_loss: 4.4138e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - loss: 4.3909e-05 - val_loss: 2.4479e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - loss: 4.5953e-05 - val_loss: 2.6581e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 45ms/step - loss: 3.4843e-05 - val_loss: 2.6013e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35

In [ ]:
# Predictions
HV_bilstm_gru_predictions = bilstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 4.627419194858454e-06
RMSE: 0.002151143694609557
MAE: 0.002051647363948764
MAPE: 8776.591474993325%


In [ ]:
# path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/New_predictions/PV/'

# df_pred = pd.DataFrame(HV_lstm_predictions[:,0], columns=['lstm_pred'])
# filename = 'LSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_gru_predictions[:,0], columns=['gru_pred'])
# filename = 'GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_lstm_gru_predictions[:,0], columns=['lstm_gru_pred'])
# filename = 'LSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_predictions[:,0], columns=['bilstm_pred'])
# filename = 'BiLSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_gru_predictions[:,0], columns=['bilstm_gru_pred'])
# filename = 'BiLSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)